<a href="https://colab.research.google.com/github/ck1972/Python-Geospatial_Model1/blob/main/6a_Comparison_of_Machine_Learning_Classifiers_in_GEE_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparing Machine Learning Classifiers in GEE: An Introductory Guide with Python
## Introduction
The script aims to compare minimum distance, k-nearest neighbor (KNN), support vector machines, decision trees, random forest,and gradient tree boosting classifiers in Google Earth Engine (GEE). We use Python in Colab.

- Requirements
To run this script, the user must have an Earth Engine account. In addition, the user must authenticate the Earth Engine Python API. See the instructions [here](https://developers.google.com/earth-engine/guides/auth).
- This script will use the [geemap](https://geemap.org) Python package to visualize, and map land cover.

Following are the steps to compare the machine learning classifiers.

# Initialize and Authenticate Earth Engine
To get started with Google Earth Engine (GEE), you need to initialize and authenticate the Earth Engine API. Follow these steps.


First, import the Earth Engine API by importing the ee module into your Python environment. This module allows you to interact with the Earth Engine platform.

In [ ]:
# Import the EE API
import ee

Next, initialize the Earth Engine API. You must initialize the API to use Earth Engine functionalities. This involves authenticating your session and initializing the library. When you run the ee.Initialize() command for the first time, you might be prompted to authenticate your session. This will open a web browser window where you need to log in with your Google account and grant Earth Engine access.

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-kamusoko-test') # Change to your EE project

## Import Libraries
Next, import the essential libraries needed to process and analyze the datasets.

In [ ]:
# Import the necessary libraries
import geemap

## Import the project boundary
First, import the study area boundary boundary.

In [ ]:
# Load the boundary
boundary = ee.FeatureCollection('users/kamas72_ML_Zim_Cities/Bulawayo_Crop_Boundary')

# Load the boundary
training_areas = ee.FeatureCollection('users/kamas72_ML_Zim_Cities/Updated_TA_2020_Bul_May_21_GEE')

## Prepare Sentinel-2 imagery
First, preprocess Sentinel-2 imagery by first applying cloud masking to remove cloud and cirrus pixels using the QA60 band. Then load Sentinel-2 surface reflectance images within a specified date range and filters them based on cloud cover, retaining images with less than 20% cloudiness. A median composite is generated from the filtered images and clipped to a specified boundary region. Finally, the composite is visualized as a false-color image using selected bands on an interactive map, complete with layer control for enhanced usabilit

In [ ]:
# Function for Cloud Masking
def mask_s2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = ee.Number(2).pow(10).int()
    cirrusBitMask = ee.Number(2).pow(11).int()
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)

# Load Sentinel-2
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate('2024-03-01', '2024-06-30') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(mask_s2clouds) \
    .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12'])

# Clip the composite
composite = s2.median().clip(boundary)

# Print Sentinel-2 Composite
print('Sentinel-2 Composite:', composite.getInfo())

# Display the Sentinel-2 composite
# Initialize the map
map1 = geemap.Map()

# Add the composite image to the map with specified display settings.
map1.addLayer(composite, {'bands': ['B11', 'B8', 'B3'], 'min': 0, 'max': 0.3}, 'Sentinel-2 Composite')

# Display the map with layer control.
map1.centerObject(boundary, 12)
map1.addLayerControl()
map1

Sentinel-2 Composite: {'type': 'Image', 'bands': [{'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B7', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326'

Map(center=[-20.112185699155855, 28.554695214742292], controls=(WidgetControl(options=['position', 'transparen…

## Prepare training data
In this step, we prepare the dataset for training and testing machine learning models by processing satellite imagery and training labels. We start by selecting Sentinel-2 bands (B2 to B12) and clipping the composite image to the specified boundary region, defining the input features. Next, we rasterize the vector training data using the Cl_Id property to create a raster layer representing class labels and add it as a new band (class) to the input features. To create a representative dataset, we use stratified sampling to extract reflectance values and class labels, ensuring proportional representation across classes. A random column is added to the dataset with a fixed seed for reproducibility, allowing us to split the data into 70% for training and 30% for validation. Finally, we confirm the dataset sizes to ensure the split is as intended. This process prepares the data for effective training and validation of machine learning models.

In [ ]:
# Use ee.List for band selection
bands = ee.List(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12'])
input_features = composite.clip(boundary)
print('input features: ', input_features.getInfo())

# Rasterise training data
training_rasterized = training_areas.reduceToImage(
    properties=['Cl_Id'],
    reducer=ee.Reducer.first()
).toInt().remap([0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5]) # Bare areas, Built-up, Cropland, Grass/ open areas, Woodlands, Water

# Add a class band to features
input_features = input_features.addBands(training_rasterized.toInt().rename('class'))

# Sample the reflectance, elevation, and slope values for each training point
training_dataset = input_features.stratifiedSample(
    numPoints=10000,
    classBand="class",
    region=boundary,
    scale=20
)

# Add a random column (named random) and specify seed value for repeatability
training_dataset = training_dataset.randomColumn('random', 50)

# Split into training and testing (validation) sets
split = 0.7  # Split the training dataset into 70% training and 30% testing (validation)
Sample_training = training_dataset.filter(ee.Filter.lt('random', split))
Sample_test = training_dataset.filter(ee.Filter.gte('random', split))

# Print the training and testing (validation data)
print("Number of training pixels:", Sample_training.size().getInfo())
print("Number of validation pixels:", Sample_test.size().getInfo())

input features:  {'type': 'Image', 'bands': [{'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'B7', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': 0, 'max': 6.553500175476074}, 'crs': 'EPSG:4326', 'cr

## Function for training machine learning classifiers
Next, we define and train multiple classifiers for satellite image classification. A helper function, train_classifier, is created to train any classifier using the specified training dataset. It requires three inputs: the classifier, the training dataset, and the input bands. The function uses the train method to associate class labels (classProperty) with the selected input bands (inputProperties). Five classifiers are initialized: a minimum distance, a decision tree (smileCart), a random forest (smileRandomForest with 100 trees),a support vector machine classifier (SVM with an RBF kernel and gamma parameter of 0.5), and a gradient tree boost classifier (smileGradientTreeBoost with parameters such as shrinkage and sampling rate). These classifiers can be trained on the dataset and later used for image classification tasks.

In [ ]:
# Create a function for training classifiers
def train_classifier(classifier, training_data):
    return classifier.train(
        features=training_data,
        classProperty='class',
        inputProperties=bands
    )

md_classifier = ee.Classifier.minimumDistance() # Minimum distance
knn_classifier = ee.Classifier.smileKNN(k=5)  # K-Nearest Neighbors with k=5
rbf_svc_classifier = ee.Classifier.libsvm(kernelType='RBF', gamma=0.5) # Support vector machines
dt_classifier = ee.Classifier.smileCart() # CART decision tree
rf_classifier = ee.Classifier.smileRandomForest(100) # Random forest
gtb_classifier = ee.Classifier.smileGradientTreeBoost( # gradient tree boosting
    numberOfTrees=50,
    shrinkage=0.005,
    samplingRate=0.7,
    maxNodes=None,
    loss='LeastAbsoluteDeviation',
    seed=0
)

## Define the train and classification function
This function, train_run_classification, dynamically trains and applies a selected machine learning classifier to classify satellite imagery. It begins by initializing placeholders for the trained classifier and the classified map. Based on the selected_classifier input, the function selects one of five classifiers: Random Forest, Minimum Distance, CART (Decision Tree), SVM with an RBF kernel, or Gradient Tree Boost, each configured with appropriate parameters. The chosen classifier is trained using the train_classifier function on the provided training dataset. After training, the function classifies the input features (input_features) using the trained classifier and clips the classified map to the specified boundary. A predefined color palette and visualization parameters are prepared for displaying the classified map. The function returns the trained classifier and the classified map, allowing for both accuracy assessment and visualization. This modular design enables flexible experimentation with different classifiers for land cover or other remote sensing classification tasks.

In [ ]:
# Define a function
def train_run_classification(selected_classifier, sample_training, bands, boundary):
    trained_classifier = None
    classified_map = None
    palette = ["grey", "red", "yellow", "lime", "green", "blue"]
    viz = {"min": 0, "max": 5, "palette": palette}

    if selected_classifier == "Minimum distance":
        md_classifier = ee.Classifier.minimumDistance()
        trained_classifier = train_classifier(md_classifier, sample_training)
        classified_map = input_features.select(bands).classify(trained_classifier).clip(boundary)
    elif selected_classifier == "KNN":
        knn_classifier = ee.Classifier.smileKNN(k=5)  # Correct initialization for KNN
        trained_classifier = train_classifier(knn_classifier, sample_training)
        classified_map = input_features.select(bands).classify(trained_classifier).clip(boundary)
    elif selected_classifier == "SVM":
        rbf_svc_classifier = ee.Classifier.libsvm(kernelType='RBF', gamma=0.5)
        trained_classifier = train_classifier(rbf_svc_classifier, sample_training)
        classified_map = input_features.select(bands).classify(trained_classifier).clip(boundary)
    elif selected_classifier == "CART":
        dt_classifier = ee.Classifier.smileCart()
        trained_classifier = train_classifier(dt_classifier, sample_training)
        classified_map = input_features.select(bands).classify(trained_classifier).clip(boundary)
    elif selected_classifier == "Random forest":
        rf_classifier = ee.Classifier.smileRandomForest(100)
        trained_classifier = train_classifier(rf_classifier, sample_training)
        classified_map = input_features.select(bands).classify(trained_classifier).clip(boundary)
    elif selected_classifier == "Gradient tree boost":
        gtb_classifier = ee.Classifier.smileGradientTreeBoost(
            numberOfTrees=50, shrinkage=0.005, samplingRate=0.7, maxNodes=None, loss='LeastAbsoluteDeviation', seed=0)
        trained_classifier = train_classifier(gtb_classifier, sample_training)
        classified_map = input_features.select(bands).classify(trained_classifier).clip(boundary)
    else:
        print("Invalid classifier")

    return trained_classifier, classified_map

## Train and perform classification
Next, we train six classifiers (Minimum Distance, KNN, SVM, Decision Tree, Random Forest, and Gradient Tree Boost) using the train_run_classification function and generate classified maps for each. The training uses the stratified training dataset (Sample_training) and the specified bands, while the classified maps are clipped to the boundary region. To visualize the results, the display_map function is defined to create an interactive map using geemap. This function applies a color palette and visualization parameters to the classified map and centers the map on the boundary.

In [ ]:
# Train classifiers and get the classified maps
trainedMD, md_classified_map = train_run_classification("Minimum distance", Sample_training, bands, boundary)
trainedKNN, knn_classified_map = train_run_classification("KNN", Sample_training, bands, boundary)
trainedRBFSVC, rbf_svc_classified_map = train_run_classification("SVM", Sample_training, bands, boundary)
trainedDT, dt_classified_map = train_run_classification("CART", Sample_training, bands, boundary)
trainedRF, rf_classified_map = train_run_classification("Random forest", Sample_training, bands, boundary)
trainedGTB, gtb_classified_map = train_run_classification("Gradient tree boost", Sample_training, bands, boundary)

## Display land cover maps
Each classified map is displayed with a title corresponding to its classifier.

In [ ]:
# Define a function to add a land cover legend
def add_legend(map_instance):
    legend_dict = {
        "Bare areas": (128, 128, 128),  # grey
        "Built-up": (255, 0, 0),        # red
        "Cropland": (255, 255, 0),      # yellow
        "Grass/open areas": (0, 255, 0),# lime
        "Woodlands": (0, 128, 0),       # green
        "Water": (0, 0, 255),           # blue
    }
    map_instance.add_legend(legend_title="Land Cover Legend", legend_dict=legend_dict)

# Create and display the maps for each classifier with legends
def display_map_with_legend(classified_map, title):
    map_instance = geemap.Map()
    palette = ["grey", "red", "yellow", "lime", "green", "blue"]
    viz = {"min": 0, "max": 5, "palette": palette}
    map_instance.centerObject(boundary, 12)
    map_instance.addLayer(classified_map, viz, title)
    map_instance.addLayerControl()
    add_legend(map_instance)  # Add the legend to the map
    return map_instance

# Display each map with a legend
map_md = display_map_with_legend(md_classified_map, "Minimum distance classification")
map_knn = display_map_with_legend(knn_classified_map, "KNN classification")
map_rbf_svc = display_map_with_legend(rbf_svc_classified_map, "SVM (RBF) classification")
map_dt = display_map_with_legend(dt_classified_map, "Decision tree classification")
map_rf = display_map_with_legend(rf_classified_map, "Random forest classification")
map_gtb = display_map_with_legend(gtb_classified_map, "Gradient tree boost classification")

### Minimum distance-derived land cover map
Display the land cover map derived from the minimum distance classifier.

In [ ]:
# Display the minimum distance classifier land cover map.
map_md

Map(center=[-20.112185699155855, 28.554695214742292], controls=(WidgetControl(options=['position', 'transparen…

### KNN-derived land cover map
Display the land cover map derived from the KNN classifier.

In [ ]:
# Display the land cover map derived from the KNN classifier.
map_knn

Map(center=[-20.112185699155855, 28.554695214742292], controls=(WidgetControl(options=['position', 'transparen…

### Support vector machine-derived land cover map
Display the land cover map derived from the support vector machine (SVM) classifier.

In [ ]:
# Display the land cover map derived from the support vector machine (SVM) classifier.
map_rbf_svc

Map(center=[-20.112185699155855, 28.554695214742292], controls=(WidgetControl(options=['position', 'transparen…

### Decision tree-derived land cover map
Display the land cover map derived from the decision tree classifier.

In [ ]:
# Display the land cover map derived from the decision tree classifier.
map_dt

Map(center=[-20.112185699155855, 28.554695214742292], controls=(WidgetControl(options=['position', 'transparen…

### Random forest-derived land cover map
Display the land cover map derived from the random forest classifier.

In [ ]:
# Display the land cover map derived from the random forest classifier.
map_rf

Map(center=[-20.112185699155855, 28.554695214742292], controls=(WidgetControl(options=['position', 'transparen…

### Gradient tree boost-derived land cover map
Display the land cover map derived from the gradient tree boost classifier.

In [ ]:
# Display the land cover map derived from the gradient boosting classifier.
map_gtb

Map(center=[-20.112185699155855, 28.554695214742292], controls=(WidgetControl(options=['position', 'transparen…

## Classification accuracy assessment
Finally, we evaluate the accuracy of classifiers using a test dataset. The compute_metrics function takes a trained classifier, test dataset, and algorithm name as inputs. It applies the classifier to the test data, generating predictions, and then calculates an error matrix that compares predicted classes with reference data. Key metrics such as user accuracy, producer accuracy, and overall accuracy are extracted from the error matrix and printed alongside the algorithm name for clarity. The function is applied to each classifier (minimum distance, decision trees, random forest, RBF support vector machines classifier, and gradient tree boosting) using the test dataset (Sample_test), providing a comprehensive accuracy assessment for all models.

In [ ]:
# Function to compute accuracy metrics
def compute_metrics(classifier, validation_data, algorithm_name):
    validation_data_classified = validation_data.classify(classifier)
    error_matrix = validation_data_classified.errorMatrix('class', 'classification')
    print(f'Algorithm: {algorithm_name}')
    print('Validation Error Matrix:\n', error_matrix.getInfo())
    print('User Accuracy: ', error_matrix.consumersAccuracy().getInfo())
    print('Producer Accuracy: ', error_matrix.producersAccuracy().getInfo())
    print('Overall Accuracy: ', error_matrix.accuracy().getInfo())
    print('-----------------------------------------------')

# Compute metrics for each classifier
compute_metrics(trainedMD, Sample_test, 'Minimum distance')
compute_metrics(trainedKNN, Sample_test, 'KNN')
compute_metrics(trainedRBFSVC, Sample_test, 'RBF SVM')
compute_metrics(trainedDT, Sample_test, 'Decision trees')
compute_metrics(trainedRF, Sample_test, 'Random forest')
compute_metrics(trainedGTB, Sample_test, 'Gradient tree boosting')

Algorithm: Minimum distance
Validation Error Matrix:
 [[42, 38, 120, 94, 212, 8], [194, 293, 178, 53, 55, 0], [135, 47, 2045, 738, 29, 6], [212, 101, 580, 1330, 543, 3], [4, 0, 4, 17, 380, 65], [2, 0, 0, 6, 47, 36]]
User Accuracy:  [[0.07130730050933787, 0.6116910229645094, 0.6986675777246327, 0.5942806076854334, 0.3001579778830964, 0.3050847457627119]]
Producer Accuracy:  [[0.08171206225680934], [0.37904269081500647], [0.6816666666666666], [0.48031780426146625], [0.8085106382978723], [0.3956043956043956]]
Overall Accuracy:  0.5416830773270317
-----------------------------------------------
Algorithm: KNN
Validation Error Matrix:
 [[301, 52, 38, 99, 24, 0], [45, 594, 56, 74, 4, 0], [27, 29, 2585, 356, 2, 1], [28, 72, 319, 2266, 82, 2], [16, 1, 21, 109, 319, 4], [6, 4, 0, 2, 1, 78]]
User Accuracy:  [[0.7115839243498818, 0.7898936170212766, 0.8562437893342166, 0.7797660013764625, 0.7384259259259259, 0.9176470588235294]]
Producer Accuracy:  [[0.585603112840467], [0.7684346701164295], [0.8